In [1]:
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.applications import EfficientNetB4
from utils import *

## Loading

In [2]:
directory = 'images/final/'
width = 128
height = 128
channels = 4
image_size = (width, height)
input_shape = (width, height, channels)
batch_size = 32

In [3]:
train_ds, val_ds, class_names = load_data(directory, batch_size, image_size)

Found 13849 files belonging to 905 classes.
Using 11080 files for training.
Using 2769 files for validation.


In [5]:
plot_n_images(train_ds, 1, class_names)

tf.Tensor([0. 0. 0. 0.], shape=(4,), dtype=float32)
(128, 128, 4)


### Preprocessing

In [7]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)

channels = 3
input_shape = (width, height, channels)

In [8]:
plot_n_images(train_ds, 5, class_names)

tf.Tensor([255. 255. 255.], shape=(3,), dtype=float32)
(128, 128, 3)


tf.Tensor([255. 255. 255.], shape=(3,), dtype=float32)
(128, 128, 3)


tf.Tensor([255. 255. 255.], shape=(3,), dtype=float32)
(128, 128, 3)


tf.Tensor([255. 255. 255.], shape=(3,), dtype=float32)
(128, 128, 3)


tf.Tensor([255. 255. 255.], shape=(3,), dtype=float32)
(128, 128, 3)


In [9]:
# Define the EfficientNetB4 model
base_model = EfficientNetB4(include_top=False, input_shape=input_shape, pooling='avg', weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Define the model inputs
inputs = tf.keras.Input(shape=input_shape)

x = base_model(inputs, training=False)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs, x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 1792)             17673823  
                                                                 
 dense (Dense)               (None, 512)               918016    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 905)               232585

In [11]:
history = model.fit(train_ds, batch_size=batch_size, validation_data=val_ds, epochs=25)

347/347 [==============================] - 94s 77ms/step - loss: 6.1012 - accuracy: 0.0653 - top_k_categorical_accuracy: 0.1129 - val_loss: 5.0587 - val_accuracy: 0.1372 - val_top_k_categorical_accuracy: 0.2730


In [12]:
cnn = tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Conv2D(64, (5, 5), activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Conv2D(64, (5, 5), activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Conv2D(128, (8, 8), activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(512, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(256, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

In [13]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 59, 64)        51264     
                                                                 
 batch_normalization_1 (Batc  (None, 59, 59, 64)      

In [14]:
history_cnn = cnn.fit(train_ds, batch_size=batch_size, validation_data=val_ds, epochs=25)

347/347 [==============================] - 39s 42ms/step - loss: 6.8205 - accuracy: 0.0239 - top_k_categorical_accuracy: 0.0418 - val_loss: 6.5240 - val_accuracy: 0.0394 - val_top_k_categorical_accuracy: 0.0664


## Reporting

In [77]:
training_results = {
    "efficientnetB4": 
        {"training_time" : 1073,
            "trainable_weights" : 1281929,
            "non_trainable_weights" : 17673823,
        },
    "CNN":
        {"training_time" : 637,
            "trainable_weights" : 1307721,
            "non_trainable_weights" : 2112,
        }
}

In [78]:
import pandas as pd
df_plot = pd.DataFrame(training_results).T
print(df_plot)

                training_time  trainable_weights  non_trainable_weights
efficientnetB4           1073            1281929               17673823
CNN                       637            1307721                   2112


In [79]:
# plot weights and then training time on 2 different y axes
fig = go.Figure()
fig.add_trace(go.Bar(x=df_plot.index, y=df_plot["trainable_weights"], name="Trainable Weights"))
fig.add_trace(go.Bar(x=df_plot.index, y=df_plot["non_trainable_weights"], name="Non-Trainable Weights"))
fig.update_layout(barmode='stack', title="Trainable and Non-Trainable Weights")
fig.update_layout(width=800, height=500)
fig.show()

In [14]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'], name="Training Loss"))
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_loss'], name="Validation Loss"))
# add accuracy on different y axis
fig.update_layout(yaxis2=dict(overlaying='y', side='right', range=[0, 1], title="Accuracy"))
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['accuracy'], name="Training Accuracy", yaxis="y2"))
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_accuracy'], name="Validation Accuracy", yaxis="y2"))
fig.update_layout(title="EfficientNetB4 Loss & accuracy",
                  legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=1
                        ),
                  yaxis=dict(range=[0, 5],title="Loss"),
                  xaxis=dict(title="Epochs")
                  )
fig.update_layout(width=1000, height=600)
fig.show()

In [23]:
# same plot but give blue color to train and orange to validation
fig = go.Figure()
fig.add_trace(go.Scatter(x=history_cnn.epoch, y=history_cnn.history['loss'], name="Training Loss"))
fig.add_trace(go.Scatter(x=history_cnn.epoch, y=history_cnn.history['val_loss'], name="Validation Loss"))
# add accuracy on different y axis
fig.update_layout(yaxis2=dict(overlaying='y', side='right', range=[0, 1], title="Accuracy"))
fig.add_trace(go.Scatter(x=history_cnn.epoch, y=history_cnn.history['accuracy'], name="Training Accuracy", yaxis="y2"))
fig.add_trace(go.Scatter(x=history_cnn.epoch, y=history_cnn.history['val_accuracy'], name="Validation Accuracy", yaxis="y2"))
fig.update_layout(title="CNN Loss & Accuracy",
                  legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=1
                        ),
                  yaxis=dict(range=[0, 5],title="Loss"),
                  xaxis=dict(title="Epochs")
                  )
fig.update_layout(width=1000, height=600)
fig.show()

## TEST

In [15]:
def top_k(y_pred, k=3):
    # retrieve the top k predictions as sorted dictionary of {class: probability}
    top_k = tf.math.top_k(y_pred, k=k)
    prob = top_k.values.numpy().tolist()
    prob = [round(p, 2) for p in prob]
    # convert the dictionary to a list of class indices
    name = top_k.indices.numpy().tolist()
    name = [class_names[i] for i in name]
    ret = {}
    for i in range(len(prob)):
        if prob[i] > 0.05:
            ret[name[i]] = prob[i]
    return ret

In [16]:
y_pred_cnn = cnn.predict(val_ds)
y_pred_eff = model.predict(val_ds)

87/87 [==============================] - 8s 50ms/step


In [17]:
y_test = []
for i in val_ds:
    y_test.extend(i[1].numpy())
import numpy as np
y_test = np.array(y_test)

In [18]:
y_pred_cnn_ = np.argmax(y_pred_cnn, axis=1).reshape(-1, 1)
y_pred_eff_ = np.argmax(y_pred_eff, axis=1).reshape(-1, 1)

y_test = np.argmax(y_test, axis=1).reshape(-1, 1)

In [40]:
from sklearn.metrics import cohen_kappa_score
kappa_eff = cohen_kappa_score(y_test, y_pred_eff_)
kappa_cnn = cohen_kappa_score(y_test, y_pred_cnn_)

print("Kappa score for EfficientNetB4: ", kappa_eff)
print("Kappa score for CNN: ", kappa_cnn)

Kappa score for EfficientNetB4:  0.7446011818478946
Kappa score for CNN:  0.28485547620141827


In [19]:
for image, label in val_ds.take(1):
    for i in range(10):
        plot_image(image[i], class_names[int(tf.argmax(tf.reshape(label[i], (-1, 1))))], top_k(y_pred_eff[i], k=5))
        plot_image(image[i], class_names[int(tf.argmax(tf.reshape(label[i], (-1, 1))))], top_k(y_pred_cnn[i], k=5))
        

### charmander test

In [140]:
# load images/test/charmander.png to test
test_image = tf.keras.preprocessing.image.load_img('images/test/charmander.png', color_mode='rgba', target_size=image_size)
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = tf.expand_dims(test_image, axis=0)
test_image = preprocess_image(test_image)
print(test_image.shape)

(1, 128, 128, 3)


In [147]:
pred = model.predict(test_image)

1/1 [==============================] - 0s 55ms/step


In [159]:
top_k(pred[0], k=5)

{'Scraggy': 0.51, 'Charmander': 0.26, 'Buizel': 0.12}

In [169]:
plot_image(test_image[0], "Charmander", top_k(pred[0], k=5))